# Analysis of PbSO4 refinement results
The results from least-squares, bumps-dream, and emcee will be compared and discussed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [2]:
# Read all data
data_least_squares = np.load('refinement_results/least_squares.npy', allow_pickle=True).item()
data_bumps = np.load('refinement_results/bumps_dream_steps5000_burn500_pop1.npy', allow_pickle=True).item()

In [3]:
print(data_least_squares.keys())
print(data_bumps.keys())

dict_keys(['length_a', 'length_b', 'length_c', 'scale', 'resolution_u', 'resolution_v', 'resolution_w', 'zero_shift', 'intensity', 'intensity-1'])
dict_keys(['names', 'samples', 'logp', 'parameters'])


The obtained parameters and their respective errorbars.     
There is an systematic underestimation in the errorbars from bumps-DREAM as disucssed [here](https://github.com/bumps/bumps/pull/94)

In [5]:
for name in sorted(data_least_squares.keys()):
    p1 = data_least_squares[name]
    p2 = data_bumps['parameters'][name]
    print(f'{name:15}: least-squares {p1[0]:9.4f} +- {p1[1]:8.6f} | bumps {p2[0]:9.4f} +- {p2[1]:8.6f}')

intensity      : least-squares  197.1217 +- 1.242605 | bumps  240.4465 +- 0.873294
intensity-1    : least-squares  240.0542 +- 1.605187 | bumps  196.9511 +- 0.650893
length_a       : least-squares    8.4770 +- 0.000271 | bumps    8.4769 +- 0.000151
length_b       : least-squares    5.3958 +- 0.000187 | bumps    5.3957 +- 0.000117
length_c       : least-squares    6.9558 +- 0.000319 | bumps    6.9558 +- 0.000168
resolution_u   : least-squares    0.1948 +- 0.015901 | bumps    0.1938 +- 0.009232
resolution_v   : least-squares   -0.4756 +- 0.030428 | bumps   -0.4747 +- 0.016805
resolution_w   : least-squares    0.4787 +- 0.012812 | bumps    0.4793 +- 0.006860
scale          : least-squares    1.2105 +- 0.005795 | bumps    1.2101 +- 0.003277
zero_shift     : least-squares    0.1568 +- 0.002768 | bumps    0.1578 +- 0.001670


## Analysis of posterior distributions obtained by MC simulations